In [1]:
import cv2
import json
import torch
import urllib.request
from ultralytics import YOLO
import open3d
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy.ndimage import gaussian_filter

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Parametri intrinseci della fotocamera

In [2]:
# Parametri intrinseci della fotocamera
fvideo = cv2.VideoCapture("./references/video/cardrive.mp4")

ret, fframe = fvideo.read()
fx = 500  # Lunghezza focale in pixel sull'asse x
fy = 500  # Lunghezza focale in pixel sull'asse y
cx = fframe.shape[1] // 2  # Centro ottico sull'asse x (al centro dell'immagine)
cy = fframe.shape[0] // 2  # Centro ottico sull'asse y (al centro dell'immagine)
fvideo.release()

apertura file json

In [3]:
with open('class_to_img.json', 'r') as file:
    classtoimg = json.load(file)

In [4]:

# Carica il modello YOLO preaddestrato (puoi sostituirlo con un modello addestrato su un tuo dataset)
yolo_model = YOLO('yolov8n.pt')  # Cambia con il tuo modello

# Carica un frame e rileva gli oggetti
frame = cv2.imread('bus.jpg')
results = yolo_model.track(frame)

# Visualizza i risultati

annotated_frame = results[0].plot()
print(results[0].boxes.id)

cv2.imshow("si", annotated_frame)
cv2.waitKey(0)

cv2.destroyAllWindows()


0: 640x480 4 persons, 1 bus, 1 stop sign, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inference, 45.0ms postprocess per image at shape (1, 3, 640, 480)
tensor([1., 2., 3., 4., 5., 6.])


Istanziare il modello di stima della profondita'

In [5]:

model_type = "DPT_Large"

midas = torch.hub.load("intel-isl/MiDaS", model_type)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform



Using cache found in C:\Users\Edoardo/.cache\torch\hub\intel-isl_MiDaS_master
c:\Users\Edoardo\anaconda3\envs\myenv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using cache found in C:\Users\Edoardo/.cache\torch\hub\intel-isl_MiDaS_master


In [6]:
print(torch.cuda.is_available())

True


In [7]:
video_path = "./references/video/cardrive.mp4"
cap = cv2.VideoCapture(video_path)

# Ottieni il framerate (FPS) del video
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(fps)
frame_time = int(1000/fps)
frame_count = 0


while cap.isOpened():

    # Leggi il frame corrente
    ret, frame = cap.read()
    if not ret:
        break  # Esci se il video è terminato
    cv2.imshow('frame', frame)

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

30


definizione della funzione midas_elaboration da inserire nel ciclo della gestione dei frame del video

In [8]:
def midas_elaboration(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_batch = transform(frame).to(device)
    
    # Elabora il frame (aggiungi qui il tuo codice di elaborazione)
    
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=frame.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    output = prediction.cpu().numpy()
    output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
    output_normalized = output_normalized.astype(np.uint8)
    #cv2.imshow('frame', output_normalized)
    return output_normalized

definizione della funzione yolo_elaboration da inserire nel ciclo della gestione dei frame del video

In [9]:
def yolo_elaboration(frame):
    detection_info = []
    #carica l'immagine
    #img = cv2.imread(frame)

    #esegui il rilevamento con yolov8
    yolo_results = yolo_model.track(frame, persist=True)

    #ottieni i risultati del rilevamento (results[0] indica le informazioni ottenute dall'unica immagine in input, se ci fossero piu di una immagine allora sarebbe results[i] con i = 1...n)
    detection = yolo_results[0].boxes.xyxy #coordinate della bounding box
    confidences = yolo_results[0].boxes.conf #confidenza della rilevazione
    class_ids = yolo_results[0].boxes.cls #ID delle classi di appartenenza dell'oggetto
    obj_ids = yolo_results[0].boxes.id #ID dell'oggetto

    #visualizza i risultati
    for i, (box, conf, cls_id, obj_id) in enumerate(zip(detection, confidences, class_ids, obj_ids)):
        x1, y1, x2, y2 = map(int, box) #estrai le coordinate della bounding box

        label = f"{yolo_model.names[int(cls_id)]} ({conf:.2f})" #viene creata un'etichetta contenente il nome della classe (poiche class_ids e' un intero che grazie a model.names[cls_id] viene convertito nella corrispondente classe) e la confidenza

        detection_info.append({
            #'frame': capture.get(cv2.CAP_PROP_POS_FRAMES),
            'id': obj_id,
            'label': label,
            'confidence': conf,
            'x1': x1,
            'x2': x2,
            'y1': y1,
            'y2': y2
        })
    annotated_frame = yolo_results[0].plot()
    cv2.imshow("si", annotated_frame)
    return detection_info

combinazione dei risultati delle due funzioni precedenti per calcolare la distanza stimata degli oggetti rilevati nel frame

In [10]:
def elaboration(midas, yolo, coefficiente, old_detection_info):
    detection_info = yolo
    depth_map = midas
    npframe = np.array(depth_map)
    updated_detection_info = []
    for object in detection_info:
        obj_id = object['id'] 
        label = object['label']
        confidence = object['confidence']
        x1 = object['x1']
        x2 = object['x2']
        y1 = object['y1']
        y2 = object['y2']
        if confidence < 0.50:
            continue
        object_data = []
        for y in range(y1, y2):
            for x in range(x1, x2):
                depth = depth_map[y,x]
                real_depth = coefficiente / depth
                real_x = ((x - cx) * real_depth)/fx
                real_y = ((y - cy) * real_depth)/fy
                object_data.append((real_x, real_y, real_depth))
        updated_detection_info.append({
            'id' : obj_id,
            'label' : label,
            'confidence' : confidence,
            '3dcoord' : object_data
        })
    return updated_detection_info
        


definizione della funzione threedMapConversion da inserire nel ciclo della gestione dei frame del video per inserire gli oggetti rilevati in una mappa 3d

In [43]:
def threedMapConversion(info, detected):
    points = []
    for object in info:
        
        ##obj_class = object['label']
        ##img = classtoimg.get(obj_class, "Percorso non trovato")
        threedcoords = object['3dcoord']
        points.extend(threedcoords)
    points = np.array(points)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    ##o3d.visualization.draw_geometries([pcd])
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)
    ctr = vis.get_view_control()
    parameters = o3d.io.read_pinhole_camera_parameters("ScreenCamera_2024-11-26-20-48-41.json")
    ctr.convert_from_pinhole_camera_parameters(parameters, True)
    vis.run()
    vis.destroy_window()

ciclo di gestione dei frame del video

In [45]:
video_path = "./references/video/cardrive.mp4"
cap = cv2.VideoCapture(video_path)

# Ottieni il framerate (FPS) del video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_to_elaborate = int(fps / 4)
frame_time = int(1000/fps)
frame_count = 0
coefficiente = 100

while cap.isOpened():

    # Leggi il frame corrente
    ret, frame = cap.read()
    if not ret:
        break  # Esci se il video è terminato
    # Processa solo 1 frame ogni secondo
    if frame_count % frame_to_elaborate == 0:
        #parte MiDaS
        midas_output = midas_elaboration(frame)
        print("parte midas completata")
        #parte YOLO
        yolo_output = yolo_elaboration(frame)
        print("parte yolo completata")
        #fu-sio-nee
        general_output = elaboration(midas_output, yolo_output, coefficiente, 4)
        print("parte generale completata")
        """
        debug
        print(general_output)
        """
        #mappa 3D dell'ambiente circostante
        threedMapConversion(general_output, 0)
        
    frame_count += 1

    if cv2.waitKey(15) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

parte midas completata

0: 384x640 4 cars, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 4 cars, 5.5ms
Speed: 1.0ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 5 cars, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 6 cars, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 7 cars, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 

C:\Users\Edoardo\AppData\Local\Temp\ipykernel_8900\144017356.py:20: RuntimeWarning: divide by zero encountered in scalar divide
  real_depth = coefficiente / depth


parte midas completata

0: 384x640 3 cars, 1 truck, 6 traffic lights, 6.5ms
Speed: 1.5ms preprocess, 6.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 4 cars, 7 traffic lights, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 4 cars, 6 traffic lights, 5.5ms
Speed: 0.5ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
parte midas completata

0: 384x640 3 cars, 5 traffic lights, 5.5ms
Speed: 1.5ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
parte yolo completata
parte generale completata
